# 计算所有方法的平均带宽并且和 dmc_620 baseline 对比

基于HN-F同步节点数据的带宽

In [1]:
def cmn_mc_avg(perf_raw_data_CMN):
    perf_agg_data_CMN = perf_raw_data_CMN.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()
    perf_agg_data_CMN['type'] = perf_agg_data_CMN['event'].apply(
        lambda x: 'reqs' if 'reqs'in x else 'retries'
    )
    perf_agg_data_CMN['socket'] = perf_agg_data_CMN['event'].apply(
        lambda x: 0 if x[8] == '0' else '1'
    )

    perf_agg_data_CMN = perf_agg_data_CMN.drop('event', axis=1)

    perf_agg_data_CMN.loc[perf_agg_data_CMN["type"] == "retries", "value"] *= -1

    bw_all_CMN = perf_agg_data_CMN.groupby(['timestamp']).agg(value=('value', 'sum')).reset_index()

    bw_all_CMN['interval'] = bw_all_CMN['timestamp'].diff()
    bw_all_CMN.loc[0, 'interval'] = bw_all_CMN.loc[0, 'timestamp']

    bw_all_CMN['memory bandwidth (GB/s)'] = (bw_all_CMN['value'] * 64) / (bw_all_CMN['interval'] * 1000000000)

    return bw_all_CMN['memory bandwidth (GB/s)'][0]

处理dmc_620内存控制器得到的数据

In [3]:
def dmc_620_avg(perf_raw_data_DMC):
    perf_agg_data_DMC = perf_raw_data_DMC.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()
    perf_agg_data_DMC['type'] = perf_agg_data_DMC['event'].apply(
        lambda x: 'read' if x[-2:-1] == '0' else 'write'
    )
    perf_agg_data_DMC['device_id'] = perf_agg_data_DMC['event'].apply(
        lambda x: x[11:20]
    )
    perf_agg_data_DMC['socket'] = perf_agg_data_DMC['device_id'].apply(
        lambda x: 0 if x[0] == '1' else '1'
    )

    perf_agg_data_DMC = perf_agg_data_DMC.drop('event', axis=1)

    bw_all_DMC = perf_agg_data_DMC.groupby(['timestamp']).agg(value=('value', 'sum')).reset_index()

    bw_all_DMC['interval'] = bw_all_DMC['timestamp'].diff()
    bw_all_DMC.loc[0, 'interval'] = bw_all_DMC.loc[0, 'timestamp']

    bw_all_DMC['memory bandwidth (GB/s)'] = (bw_all_DMC['value'] * 64) / (bw_all_DMC['interval'] * 1000000000)

    return bw_all_DMC['memory bandwidth (GB/s)'][0]

基于watch point计算平均带宽的方式

In [4]:
def watch_point_avg(perf_raw_data_WC):
    perf_agg_data_WC = perf_raw_data_WC.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()

    perf_agg_data_WC = perf_agg_data_WC.drop('event', axis=1)

    bw_all_WC = perf_agg_data_WC.groupby(['timestamp']).agg(value=('value', 'sum')).reset_index()

    bw_all_WC['interval'] = bw_all_WC['timestamp'].diff()
    bw_all_WC.loc[0, 'interval'] = bw_all_WC.loc[0, 'timestamp']

    bw_all_WC['memory bandwidth (GB/s)'] = (bw_all_WC['value'] * 32) / (bw_all_WC['interval'] * 1000000000)

    return bw_all_WC['memory bandwidth (GB/s)'][0]

最后是LLC的

In [5]:
def llc_avg(perf_raw_data_LLC):
    perf_agg_data_LLC = perf_raw_data_LLC.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()

    bw_all_CMN = perf_agg_data_LLC.groupby(['timestamp']).agg(value=('value', 'sum')).reset_index()

    bw_all_CMN['interval'] = bw_all_CMN['timestamp'].diff()
    bw_all_CMN.loc[0, 'interval'] = bw_all_CMN.loc[0, 'timestamp']

    bw_all_CMN['memory bandwidth (GB/s)'] = (bw_all_CMN['value'] * 64) / (bw_all_CMN['interval'] * 1000000000)

    return bw_all_CMN['memory bandwidth (GB/s)'][0]

补充基于llc_uncore的

In [6]:
def llc_uncore_avg(perf_raw_data_LLCU):
    perf_agg_data_CMN = perf_raw_data_LLCU.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()

    perf_agg_data_CMN['socket'] = perf_agg_data_CMN['event'].apply(
        lambda x: 0 if x[8] == '0' else '1'
    )

    perf_agg_data_CMN = perf_agg_data_CMN.drop('event', axis=1)

    bw_all_CMN = perf_agg_data_CMN.groupby(['timestamp']).agg(value=('value', 'sum')).reset_index()

    bw_all_CMN['interval'] = bw_all_CMN['timestamp'].diff()
    bw_all_CMN.loc[0, 'interval'] = bw_all_CMN.loc[0, 'timestamp']

    bw_all_CMN['memory bandwidth (GB/s)'] = (bw_all_CMN['value'] * 64) / (bw_all_CMN['interval'] * 1000000000)

    return bw_all_CMN['memory bandwidth (GB/s)'][0]

现在要做的事情就是计算所有的平均值.
首先我们要把一个benchmark的几段数据合成一段。

In [7]:
import os

DMC_csv_dir = "../data/csv/dmc_620/"
CMN_csv_dir = "../data/csv/cmn_mc/"
WP_csv_dir = "../data/csv/watch_point/"
LLC_csv_dir = "../data/csv/llc/"
LLCU_csv_dir = "../data/csv/llc_uncore/"

CMN_files = os.listdir(CMN_csv_dir)
print(CMN_files)

DMC_files = os.listdir(DMC_csv_dir)
print(DMC_files)

WP_files = os.listdir(WP_csv_dir)
print(WP_files)

LLC_files = os.listdir(LLC_csv_dir)
print(LLC_files)

LLCU_files = os.listdir(LLCU_csv_dir)
print(LLCU_files)

['544.nab_r_iter0_1732800554.csv', '503.bwaves_r_iter0_1732758672.csv', '525.x264_r_iter0_1732745373.csv', '525.x264_r_iter0_1732745249.csv', '538.imagick_r_iter0_1732798344.csv', '510.parest_r_iter0_1732770298.csv', '557.xz_r_iter0_1732752562.csv', '511.povray_r_iter0_1732775545.csv', '541.leela_r_iter0_1732748533.csv', '500.perlbench_r_iter0_1732727561.csv', '503.bwaves_r_iter0_1732759320.csv', '531.deepsjeng_r_iter0_1732746710.csv', '520.omnetpp_r_iter0_1732739475.csv', '557.xz_r_iter0_1732752994.csv', '508.namd_r_iter0_1732765498.csv', '502.gcc_r_iter0_1732730562.csv', '502.gcc_r_iter0_1732730427.csv', '521.wrf_r_iter0_1732789542.csv', '502.gcc_r_iter0_1732730308.csv', '549.fotonik3d_r_iter0_1732807437.csv', '502.gcc_r_iter0_1732730115.csv', '500.perlbench_r_iter0_1732727453.csv', '548.exchange2_r_iter0_1732750423.csv', '526.blender_r_iter0_1732793389.csv', '507.cactuBSSN_r_iter0_1732763763.csv', '523.xalancbmk_r_iter0_1732743238.csv', '557.xz_r_iter0_1732752795.csv', '503.bwaves_r

In [8]:
CMN_bench = [file for file in CMN_files if file.startswith("500")]
DMC_bench = [file for file in DMC_files if file.startswith("500")]
WP_bench = [file for file in WP_files if file.startswith("500")]
LLC_bench = [file for file in LLC_files if file.startswith("500")]
LLCU_bench = [file for file in LLC_files if file.startswith("500")]

print(CMN_bench)
print(DMC_bench)
print(WP_bench)

['500.perlbench_r_iter0_1732727561.csv', '500.perlbench_r_iter0_1732727453.csv', '500.perlbench_r_iter0_1732727231.csv']
['500.perlbench_r_iter0_1732726864.csv', '500.perlbench_r_iter0_1732726637.csv', '500.perlbench_r_iter0_1732726973.csv']
['500.perlbench_r_iter0_1732728160.csv', '500.perlbench_r_iter0_1732728050.csv', '500.perlbench_r_iter0_1732727831.csv']


In [12]:
import pandas as pd
WP_datas = []
for file in WP_bench:
    perf_raw_data_WP = pd.read_csv(WP_csv_dir + file,
                                sep='|',
                                header=None, 
                                names=["timestamp", "value", "event"], 
                                usecols=[0, 1, 3])
    perf_agg_data_WP = perf_raw_data_WP.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()
    WP_datas.append(perf_agg_data_WP)

# for data in WP_datas:
#     print(len(data))

df_concatenated = pd.concat(WP_datas, axis=0)

bench_perf_data = df_concatenated.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'sum')).reset_index()

print("watch point: ", watch_point_avg(bench_perf_data))
# print("testing benchmark 500:")

# print("llc: ", llc_avg(bench_perf_data))
# print("llc_uncore: ", llc_uncore_avg(bench_perf_data))
# print("cmn_mc: ", cmn_mc_avg(bench_perf_data))
# print("dmc_620: ", dmc_620_avg(bench_perf_data))

watch point:  60.69972793711085


In [13]:
benches = list(set([fileName.split('.')[0] for fileName in CMN_files]))
benches.sort()

methods = [{"data_path": WP_csv_dir, "func": watch_point_avg, "name": "Watch_Point"},
           {"data_path": CMN_csv_dir, "func": cmn_mc_avg, "name": "CMN_MC"},
           {"data_path": DMC_csv_dir, "func": dmc_620_avg, "name": "DMC_620"},
           {"data_path": LLC_csv_dir, "func": llc_avg, "name": "LLC_core"},
           {"data_path": LLCU_csv_dir, "func": llc_uncore_avg, "name": "LLC_uncore"}]

chart = pd.DataFrame(columns=["DMC_620", "CMN_MC", "Watch_Point", "LLC_core", "LLC_uncore"], index=benches)

for method in methods:
    path, func, name = method["data_path"], method["func"], method["name"]
    files = os.listdir(path)
    for bench in benches:
        bench_files = [file for file in files if file.startswith(bench)]
        datas = []
        for file in bench_files:
            perf_raw_data = pd.read_csv(path + file,
                                        sep='|',
                                        header=None, 
                                        names=["timestamp", "value", "event"], 
                                        usecols=[0, 1, 3])
            perf_agg_data = perf_raw_data.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()
            datas.append(perf_agg_data)

        df_concatenated = pd.concat(datas, axis=0)

        bench_perf_data = df_concatenated.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'sum')).reset_index()

        avg = func(bench_perf_data)
        # print(f"bench: {bench}  method: {name}  average: {avg}")
        chart[name][bench] = avg
        
print(chart)

/tmp/ipykernel_221263/20607920.py:33: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  chart[name][bench] = avg
/tmp/ipykernel_221263/20607920.py:33: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting va

        DMC_620      CMN_MC Watch_Point    LLC_core  LLC_uncore
500   62.127059   60.576678   60.699728    63.42955   56.060627
502  244.045841  240.348524  239.995789  226.133062  227.545839
503  326.941795  325.347429  325.463302   300.88373  286.884641
505  326.819263  325.750568  326.008061  365.325114  300.745628
507  234.566557   233.10296  232.018744  190.384499  206.913897
508   54.250723   53.286737   53.128489   49.560727   43.674671
510  344.591797  344.811266  344.405728  382.542598  334.097728
511    0.130616    0.097835    0.070067    0.398714    0.112799
519  301.107189  301.158119  300.516257  176.482435  174.546954
520  209.096187  202.502326  200.336844  183.140622  158.025199
521  314.290369  313.591669  313.838508  247.185211  227.704659
523  229.576262  228.942528  228.189611  245.696258  208.720237
525   58.085374   55.334032   55.400533   52.296677   46.557771
526  113.663545  108.957709  108.669212  126.910258  103.236012
527  258.372719  248.809071  256.568892 

/tmp/ipykernel_221263/20607920.py:33: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  chart[name][bench] = avg
/tmp/ipykernel_221263/20607920.py:33: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting va

In [15]:
chart["WP_rel_err"] = (abs((chart["Watch_Point"] - chart["DMC_620"] )/ chart["DMC_620"]) * 100)
chart['WP_rel_err'] = chart['WP_rel_err'].apply(lambda x: f"{x:.2f}%")
chart["CMN_MC_rel_err"] = (abs((chart["CMN_MC"] - chart["DMC_620"] )/ chart["DMC_620"]) * 100)
chart['CMN_MC_rel_err'] = chart['CMN_MC_rel_err'].apply(lambda x: f"{x:.2f}%")
chart["LLCc_rel_err"] = (abs((chart["LLC_core"] - chart["DMC_620"] )/ chart["DMC_620"]) * 100)
chart['LLCc_rel_err'] = chart['LLCc_rel_err'].apply(lambda x: f"{x:.2f}%")
chart["LLCu_rel_err"] = (abs((chart["LLC_uncore"] - chart["DMC_620"] )/ chart["DMC_620"]) * 100)
chart['LLCu_rel_err'] = chart['LLCu_rel_err'].apply(lambda x: f"{x:.2f}%")

chart["WP_abs_err"] = abs((chart["Watch_Point"] - chart["DMC_620"] ))
chart["CMN_MC_abs_err"] = abs((chart["CMN_MC"] - chart["DMC_620"] ))
chart["LLCc_abs_err"] = abs((chart["LLC_core"] - chart["DMC_620"] ))
chart["LLCu_abs_err"] = abs((chart["LLC_uncore"] - chart["DMC_620"] ))
print(chart)


        DMC_620      CMN_MC Watch_Point    LLC_core  LLC_uncore WP_rel_err  \
500   62.127059   60.576678   60.699728    63.42955   56.060627      2.30%   
502  244.045841  240.348524  239.995789  226.133062  227.545839      1.66%   
503  326.941795  325.347429  325.463302   300.88373  286.884641      0.45%   
505  326.819263  325.750568  326.008061  365.325114  300.745628      0.25%   
507  234.566557   233.10296  232.018744  190.384499  206.913897      1.09%   
508   54.250723   53.286737   53.128489   49.560727   43.674671      2.07%   
510  344.591797  344.811266  344.405728  382.542598  334.097728      0.05%   
511    0.130616    0.097835    0.070067    0.398714    0.112799     46.36%   
519  301.107189  301.158119  300.516257  176.482435  174.546954      0.20%   
520  209.096187  202.502326  200.336844  183.140622  158.025199      4.19%   
521  314.290369  313.591669  313.838508  247.185211  227.704659      0.14%   
523  229.576262  228.942528  228.189611  245.696258  208.720237 

In [17]:
chart.to_excel('average_bw.xlsx')